## Server run notebook

This is not the main notebook. See `instructions.ipynb`.

I'll use this notebook to record and check things along the way.

In [1]:
!nvidia-smi

Mon Dec 22 15:45:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:61:00.0 Off |                    0 |
| N/A   39C    P0             72W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## hit this problem

Doing the first test run. Looks like https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip is no longer accessible:

In [3]:
!tail -200 /home/ubuntu/mynanochat2/run_outputs/base_train_output_001.txt

[rank2]:     result = func(*args)
[rank2]:   File "/usr/lib/python3.10/urllib/request.py", line 643, in http_error_default
[rank2]:     raise HTTPError(req.full_url, code, msg, hdrs, fp)
[rank2]: urllib.error.HTTPError: HTTP Error 403: Forbidden
downloading https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip...
[rank7]: Traceback (most recent call last):
[rank7]:   File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
[rank7]:     return _run_code(code, main_globals, None,
[rank7]:   File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
[rank7]:     exec(code, run_globals)
[rank7]:   File "/home/ubuntu/learn-nanochat/my_nanochat/scripts/my_base_train.py", line 205, in <module>
[rank7]:     results = evaluate_model(orig_model, tokenizer, device, max_per_task=core_metric_max_per_task)
[rank7]:   File "/home/ubuntu/learn-nanochat/my_nanochat/scripts/my_base_eval.py", line 41, in evaluate_model
[rank7]:     download_file_with_lock(EVAL_BUNDLE_URL, 'eval_

### repeating test run

...after scp'ing base_eval from my laptop

logging to `base_train_output_002.txt`

In [4]:
!tail -25 /home/ubuntu/mynanochat2/run_outputs/base_train_output_002.txt

wandb:            train/lrm ▁
wandb:            train/mfu ▁
wandb:    train/tok_per_sec ▁
wandb:                   +1 ...
wandb: 
wandb: Run summary:
wandb:          core_metric -0.02267
wandb:                 step 10
wandb: total_training_flops 63331869759897600
wandb:  total_training_time 0
wandb:             train/dt 22.59089
wandb:      train/grad_norm 0.53604
wandb:           train/loss 11.09035
wandb:            train/lrm 1
wandb:            train/mfu 3.54326
wandb:    train/tok_per_sec 23207
wandb:                   +1 ...
wandb: 
wandb: 🚀 View run challenge-38-1 at: https://wandb.ai/ericsilberstein-self/my-nanochat/runs/0ie1fin2
wandb: ⭐️ View project at: https://wandb.ai/ericsilberstein-self/my-nanochat
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20251222_155927-0ie1fin2/logs
[W1222 16:01:23.002992110 AllocatorConfig.cpp:28] Warning: PYTORCH_CUDA_ALLOC_CONF is deprecated, use PYTORCH_ALLOC_CONF instead (

### confirm can load checkpoint

In [7]:
import torch
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_common import get_base_dir, autodetect_device_type, compute_init
from my_nanochat.my_checkpoint_manager import load_model

In [8]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d32', device=device, phase='eval')
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) if device_type == "cuda" else nullcontext()

Autodetected device type: cuda


/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


loading the model from /home/ubuntu/mynanochat2/base_checkpoints/d32 with step 10
Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 32, 'n_head': 16, 'n_kv_head': 16, 'n_embd': 2048}


In [9]:
with autocast_ctx:
    tokens = list(model.generate(tokenizer.encode("First take a right on Main St.", prepend=tokenizer.get_bos_token_id()), max_tokens=10))
tokens

[1006, 345, 400, 311, 4236, 3328, 281, 261, 24142, 4398]

In [10]:
tokenizer.decode(tokens)

' We are can be introduced examples of the utmost background'

that looks ok, also check wandb

look ok, do real run